### How to define and solve a friction contact problem in Siconos

Set path to siconos install

In [4]:
import sys
# Set this line according to your Siconos installation
sys.path.append("/Users/Franck/Softs/install-clang/siconos/lib/python3.7/site-packages/")

In [5]:
import siconos.fclib as fcl

In [6]:
# Check tuto.py for details
%run tuto.py

In [7]:
solver_options.dparam[0] = 1e-9

In [8]:
solver_options.dparam

array([1.00000000e-09, 4.67820132e-18, 1.00000000e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [9]:
velocity, reaction

(array([0.        , 0.96837722, 2.90513167, 0.        , 0.96837722,
        2.90513167, 0.        , 0.96837722, 2.90513167]),
 array([ 1.        , -0.03162278, -0.09486833,  1.        , -0.03162278,
        -0.09486833,  1.        , -0.03162278, -0.09486833]))

### How to read the problem from a file (.dat)

In [10]:
filename = "Capsules-i100-1090.dat"
fc3d_pb = sn.frictionContactProblemFromFile(filename)

In [11]:
fc3d_pb.M.matrix1

<siconos.numerics.SparseBlockStructuredMatrix; proxy of <Swig Object of type 'struct SparseBlockStructuredMatrix *' at 0x121d35ae0> >

### How to download fc problem from fclib (hdf5) file

Two ways of doing things : 

* (a) use fclib interface from numerics (requires fclib install + siconos compilation with -DWITH_FCLIB=ON)

* (b) straightfoward use of h5py tools 
   --> read fields in h5 file and fill numpy arrays


(a) with fclib interface

Warning : in this example, we assume a local formulation.
Check keys in h5 file (see below).


In [12]:
# Open h5 file
filename = "./Capsules-i2007-12.hdf5"
with h5py.File(filename) as h5f:
    assert 'fclib_local' in h5f

# Read into fclib and convert into a numerics problem
fcp = fcl.fclib_read_local(filename)
fc3d = sn.from_fclib_local(fcp)

In [14]:
fc3d.M.matrix1

<siconos.numerics.SparseBlockStructuredMatrix; proxy of <Swig Object of type 'struct SparseBlockStructuredMatrix *' at 0x121d35990> >

(b) straightfoward use of h5py

In [40]:
# Open h5 file
filename = "./Capsules-i2007-12.hdf5"

h5f = h5py.File(filename)
for it in h5f:
    print(it) # check fields 

fclib_local
guesses
solution


In [41]:
# Read W, q, mu, convert to numpy arrays if required
W = h5f['fclib_local']['W']
q = np.asarray(h5f['fclib_local']['vectors']['q'])
mu = np.asarray(h5f['fclib_local']['vectors']['mu'])  

# Check matrix type, assume csr
assert W['nz'][(0)] == -2


In [42]:
import scipy.sparse as ssp
# Write it into scipy sparse
indptr = W['p']
indices = np.asarray(W['i'])
x = np.asarray(W['x'])
Wmat = ssp.csr_matrix((x,indices,indptr))
Wmat.toarray() 

array([[ 2.00334045e+00, -5.49290862e-02, -1.12584711e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-5.49290862e-02,  6.96521974e+00,  2.99981220e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.12584729e-04,  2.99981149e-03,  6.96736465e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.89621260e+00, -1.77303675e-01,  6.31694721e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.76492894e-01,  3.25033431e+00,  5.20439295e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         6.31930714e-01,  5.19821128e-01,  1.63310907e+00]])

In [48]:
# Read solutions
reaction = np.asarray(h5f['solution']['r'])
velo = np.asarray(h5f['solution']['u'])

In [71]:
# Build numerics problem
fc3d_3 = sn.FrictionContactProblem(3, mu.shape[0], Wmat, q, mu)

In [55]:
# Close hdf5 file
h5f.close()

## Python profiling tools

https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html

https://lothiraldan.github.io/2018-02-18-python-line-profiler-without-magic/

https://github.com/rkern/line_profiler

In [50]:
%load_ext line_profiler

Examples of usage ...

In [65]:
def solve(pb, reaction, velocity, solver_options):
    sn.fc3d_driver(pb, reaction, velocity, solver_options)
    
def build_and_solve_fc(pb):
    nc = pb.numberOfContacts
    dim = pb.dimension
    velocity = np.zeros(dim * nc, dtype=np.float64)
    reaction = np.zeros_like(velocity)

    solver_options = sn.SolverOptions(sn.SICONOS_FRICTION_3D_NSGS)#sn.SICONOS_FRICTION_3D_FPP)
    eps = np.finfo(np.float64).eps
    solver_options.dparam[0] = 100 * eps
    solve(pb, reaction, velocity, solver_options)
    
def run(filename):
    fcp = fcl.fclib_read_local(filename)
    fc3d = sn.from_fclib_local(fcp)
    build_and_solve_fc(fc3d)




In [66]:
filename = "Capsules-i2007-12.hdf5"

In [67]:
%timeit run(filename)

961 µs ± 18.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [69]:
%lprun -f  build_and_solve_fc run(filename)

Timer unit: 1e-06 s

Total time: 0.000585 s
File: <ipython-input-65-d1bc6d9e636c>
Function: build_and_solve_fc at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def build_and_solve_fc(pb):
     5         1          3.0      3.0      0.5      nc = pb.numberOfContacts
     6         1          2.0      2.0      0.3      dim = pb.dimension
     7         1         13.0     13.0      2.2      velocity = np.zeros(dim * nc, dtype=np.float64)
     8         1         25.0     25.0      4.3      reaction = np.zeros_like(velocity)
     9                                           
    10         1         14.0     14.0      2.4      solver_options = sn.SolverOptions(sn.SICONOS_FRICTION_3D_NSGS)#sn.SICONOS_FRICTION_3D_FPP)
    11         1         11.0     11.0      1.9      eps = np.finfo(np.float64).eps
    12         1          9.0      9.0      1.5      solver_options.dparam[0] = 100 * eps
    13         1        508.0  